In [20]:
import pandas as pd
from jinja2 import Environment, FileSystemLoader
import datetime
import os

def load_customer_data(demographics_path, model_output_path):
    """Loads customer demographics and model output data, merging them on Customer_ID."""
    # Load CSV files
    demographics_df = pd.read_csv(demographics_path)
    model_output_df = pd.read_csv(model_output_path)
    
    # Filter customers where customer_response == 1
    model_output_df = model_output_df[model_output_df['customer_response_in_binary'] == 1]
    
    # Merge data on Customer_ID
    merged_df = pd.merge(model_output_df, demographics_df, on='Customer_ID', how='inner')
    
    # Standardize marital status
    merged_df['Marital_Status'] = merged_df['Marital_Status'].apply(lambda x: 'Married' if x == 'Married' else 'Single')
    
    return merged_df

def get_loan_image(gender, marital_status, loan_type):
    """Returns the appropriate loan image based on gender, marital status, and loan type."""
    mapping = {
        "auto": {
            "Female": "/Users/raghav/Downloads/MaleAutoLoan.png",
            "Male": "/Users/raghav/Downloads/FemaleAutoLoan.png",
            "Married": "/Users/raghav/Downloads/FamilyAutoLoan.png"
        },
        "home": {
            "Female": "/Users/raghav/Downloads/MaleHomeLoan.png",
            "Male": "/Users/raghav/Downloads/FemaleHomeLoan.png",
            "Married": "/Users/raghav/Downloads/FamilyHomeLoan.png"
        }
    }
    return mapping.get(loan_type.lower(), {}).get(marital_status if marital_status == "Married" else gender, "DefaultLoan.png")

def generate_loan_letter(customer_data, template_path):
    """Generates a home loan letter using a template and customer data."""
    try:
        # Load Jinja2 template
        template_dir = os.path.dirname(template_path)
        template_file = os.path.basename(template_path)
        env = Environment(loader=FileSystemLoader(template_dir))
        template = env.get_template(template_file)
        
        # Render template
        html_letter = template.render(customer_data)
        return html_letter
    except Exception as e:
        return f"Error rendering template: {e}"

def main():
    demographics_path = "/Users/raghav/Desktop/Apexon/Product_Marketing_Campaign/customerDemographics.csv"
    model_output_path = "/Users/raghav/Desktop/Apexon/Product_Marketing_Campaign/model_output_data.csv"
    merged_data_path = "/Users/raghav/Downloads/merged_data.csv"  # New path for merged_data.csv
    template_path = "/Users/raghav/Downloads/home_loan_template.html"
    output_dir = "/Users/raghav/Downloads/generated_template/"
    os.makedirs(output_dir, exist_ok=True)
    
    customer_df = load_customer_data(demographics_path, model_output_path)
    
    # Load merged data containing email headers and bodies
    merged_data_df = pd.read_csv(merged_data_path)
    
    for _, row in customer_df.iterrows():
        customer_id = row['Customer_ID']
        gender = row['Gender']
        marital_status = row['Marital_Status']
        
        # Get the corresponding email_header and email_body for the customer
        email_data = merged_data_df[merged_data_df['Customer_ID'] == customer_id]
        
        # Assuming merged_data.csv has columns 'email_header' and 'email_body'
        if not email_data.empty:
            email_header = email_data.iloc[0]['Header']
            email_body = email_data.iloc[0]['Email Body']
        else:
            email_header = f"Home Loan Offer for {row['Full_Name']}"
            email_body = f"Dear {row['Full_Name']},\n\nWe are pleased to offer you a home loan of $25,000. Please find the details below..."
        
        # Define loan type (assuming home loan for this example, can be dynamic)
        loan_type = "home"
        home_loan_image_path = get_loan_image(gender, marital_status, loan_type)
        
        # Prepare customer data with email body and header from merged_data.csv
        customer_data = {
            "Customer_ID": customer_id,
            "home_loan_image_path": home_loan_image_path,
            "bank_logo_path": "/Users/raghav/Downloads/newsletter_template/ally_bank_image.jpeg",
            "bank_name": "Ally Bank",
            "bank_street": "123 Main St",
            "bank_city": "Anytown",
            "bank_state": "CA",
            "bank_zip": "91234",
            "current_date": datetime.date.today().strftime("%B %d, %Y"),
            "customer_name": row['Full_Name'],
            "loan_amount": "$25,000",
            "interest_rate": "4.9%",
            "loan_term": "60",
            "monthly_payment": "$470",
            "bank_phone": "555-1212",
            "bank_email": "info@Ally.com",
            "bank_website": "www.Ally.com",
            "email_body": email_body,
            "email_header": email_header
        }
        
        # Generate loan letter
        html_output = generate_loan_letter(customer_data, template_path)
        if "Error:" not in html_output:
            output_filename = os.path.join(output_dir, f"generated_home_loan_{customer_id}.html")
            with open(output_filename, "w") as outfile:
                outfile.write(html_output)
            print(f"Loan letter generated successfully: {output_filename}")
        else:
            print(f"Error for Customer_ID {customer_id}: {html_output}")

if __name__ == "__main__":
    main()


Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_77315.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_57307.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_31251.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_94790.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_90107.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_49751.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_45147.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_71264.html
Loan letter generated successfully: /Users/raghav/Downloads/generated_template/generated_home_loan_14226.html
Loan lette